In [6]:
import json

# Load the JSON data from data.json
with open('data/smoothie_friendly_foods.json') as f:
    data = json.load(f)



In [7]:
food_names = list(map(lambda f: f['name'], data['foods']))
food_names

['Peanut Butter, smooth',
 'Cottage cheese(Milsani, tawrog chudy | Klinek)',
 'Bananas']

In [8]:
nutrients = {}
nutrients_types = set()
for food in data['foods']:
    print(food)
    food_name = food['name']
    nutrients[food_name] = {}
    for nutrient in food['nutrients']:
        nutrients[food_name][nutrient] = food['nutrients'][nutrient]['value']
        nutrients_types.add(nutrient)

nutrients

{'name': 'Peanut Butter, smooth', 'nutrients': {'calories': {'value': 588, 'measurement': 'kkal', 'sources': ['https://www.nutritionvalue.org/Peanut_Butter%2C_smooth_nutritional_value.html?size=100+g']}, 'carbohydrates': {'value': 24, 'measurement': 'g', 'sources': ['https://www.nutritionvalue.org/Peanut_Butter%2C_smooth_nutritional_value.html?size=100+g']}, 'protein': {'value': 22, 'measurement': 'g', 'sources': ['https://www.nutritionvalue.org/Peanut_Butter%2C_smooth_nutritional_value.html?size=100+g']}, 'fat': {'value': 50, 'measurement': 'g', 'sources': ['https://www.nutritionvalue.org/Peanut_Butter%2C_smooth_nutritional_value.html?size=100+g']}, 'saturated_fat': {'value': 9.5, 'measurement': 'g', 'sources': ['https://www.nutritionvalue.org/Peanut_Butter%2C_smooth_nutritional_value.html?size=100+g']}, 'fiber': {'value': 5.7, 'measurement': 'g', 'sources': ['https://www.nutritionvalue.org/Peanut_Butter%2C_smooth_nutritional_value.html?size=100+g']}}}
{'name': 'Cottage cheese(Milsani

{'Peanut Butter, smooth': {'calories': 588,
  'carbohydrates': 24,
  'protein': 22,
  'fat': 50,
  'saturated_fat': 9.5,
  'fiber': 5.7},
 'Cottage cheese(Milsani, tawrog chudy | Klinek)': {'calories': 90,
  'protein': 18,
  'carbohydrates': 3.6,
  'fat': 0.4,
  'saturated_fat': 0.3,
  'fiber': 0},
 'Bananas': {'calories': 89,
  'carbohydrates': 23,
  'protein': 1.1,
  'fat': 0.3,
  'saturated_fat': 0.1,
  'fiber': 2.6}}

In [9]:
# nutrients['Cooked buckwheat groats'].get('calories', 0)

In [10]:
import pulp as pl
import copy
def find_infeasible_constraints(prob, constraints):
    for name, constraint in constraints.items():
        test_prob = copy.deepcopy(prob)
        test_prob.constraints.pop(name)
        test_prob.solve()
        if pl.LpStatus[test_prob.status] != 'Infeasible':
            print(f"Constraint {name} is likely causing infeasibility")
        # else:
        #     print(f"Constraint {name} is not causing infeasibility")

prob = pl.LpProblem("Meal_Planning_Problem", pl.LpMinimize)
food_vars = pl.LpVariable.dicts("Amount", food_names, lowBound=0)
prob += pl.lpSum([food_vars[f] for f in food_names]), "Total Weight of Food"

calories = 2500
prob += pl.lpSum([(nutrients[f].get('calories', 0) * food_vars[f]) for f in food_names] 
                    ) >= calories, "Min Calories"

def add_nutrient_constraints(prob, nutrients, food_vars, nutrient_name, min_value, max_value):
    if nutrient_name not in nutrients_types:
        raise ValueError(f"Nutrient {nutrient_name} not found in the data")
    # Adding constraints for minimum nutrient values, if specified
    if min_value is not None:
        prob += pl.lpSum([(nutrients[f].get(nutrient_name, 0) * food_vars[f]) for f in food_names] 
                         ) >= min_value, f"Min {nutrient_name}"
    if max_value is not None:
        prob += pl.lpSum([(nutrients[f].get(nutrient_name, 0) * food_vars[f]) for f in food_names] 
                         ) <= max_value, f"Max {nutrient_name}"

add_nutrient_constraints(prob, nutrients, food_vars, 'carbohydrates', calories * 0.45 / 4,  calories * 0.65 / 4)
add_nutrient_constraints(prob, nutrients, food_vars, 'protein', calories * 0.1 / 4,  calories * 0.35 / 4)
add_nutrient_constraints(prob, nutrients, food_vars, 'fat', calories * 0.0 / 9,  calories * 0.3 / 9)
add_nutrient_constraints(prob, nutrients, food_vars, 'saturated_fat', 0,  calories * 0.1 / 9)
# add_nutrient_constraints(prob, nutrients, food_vars, 'vitamin_d', 15, 100) # mcg
# add_nutrient_constraints(prob, nutrients, food_vars, 'iron', 8, 45) # mg
# add_nutrient_constraints(prob, nutrients, food_vars, 'calcium', 1000, 2500) # mg
# add_nutrient_constraints(prob, nutrients, food_vars, 'vitamin_c', 90, 2000) # mg
# add_nutrient_constraints(prob, nutrients, food_vars, 'vitamin_a', 900, 3000) # mcg
add_nutrient_constraints(prob, nutrients, food_vars, 'fiber', 38, None) # g
# add_nutrient_constraints(prob, nutrients, food_vars, 'vitamin_e', 15, 1000) # mg
# add_nutrient_constraints(prob, nutrients, food_vars, 'vitamin_k', 120, None) # mcg
# add_nutrient_constraints(prob, nutrients, food_vars, 'selenium', 55, 400) # mcg

prob.solve()
print("Status:", pl.LpStatus[prob.status])
for v in prob.variables():
    if v.varValue > 0:
        formatted_name = v.name.replace("Amount_", "").replace("_", " ")
        print(f"{formatted_name} = {v.varValue * 100} grams")
# Print nutrients for the optimal solution

for nutrient in nutrients_types:
    nutrient_value = pl.lpSum([(nutrients[f].get(nutrient, 0) * food_vars[f].varValue) for f in food_names])
    print(f"{nutrient}: {nutrient_value}")


# Collect all constraints
constraints = {name: prob.constraints[name] for name in prob.constraints.keys()}

# Find infeasible constraints
find_infeasible_constraints(prob, constraints)
# https://ecosupplements.pl/product/norweski-olej-z-watroby-dorsza-1000mg-omega-3-bez-smaku-250-ml/

Status: Optimal
Bananas = 1571.5185999999999 grams
Cottage cheese(Milsani, tawrog chudy | Klinek) = 207.26835 grams
Peanut Butter, smooth = 155.57941 grams
carbohydrates: 406.249997
calories: 2499.9999998
fat: 83.3333342
protein: 88.8224778
fiber: 49.72750997
saturated_fat: 16.9733676
Constraint Min_Calories is likely causing infeasibility
Constraint Min_carbohydrates is likely causing infeasibility
Constraint Max_carbohydrates is likely causing infeasibility
Constraint Min_protein is likely causing infeasibility
Constraint Max_protein is likely causing infeasibility
Constraint Min_fat is likely causing infeasibility
Constraint Max_fat is likely causing infeasibility
Constraint Min_saturated_fat is likely causing infeasibility
Constraint Max_saturated_fat is likely causing infeasibility
Constraint Min_fiber is likely causing infeasibility


Included Nutrients

Vitamin D
Iron
Calcium
Vitamin C
Vitamin A
Fiber
Vitamin E
Vitamin K
Selenium






B Vitamins:
B1 (Thiamine): 1.1-1.2 mg/day
B2 (Riboflavin): 1.1-1.3 mg/day
B3 (Niacin): 14-16 mg/day
B5 (Pantothenic Acid): 5 mg/day
B6 (Pyridoxine): 1.3-1.7 mg/day
B7 (Biotin): 30 µg/day
B9 (Folate): 400 µg/day
B12 (Cobalamin): 2.4 µg/day
Minerals:

Calcium: 1,000-1,300 mg/day
Iron: 8-18 mg/day
Magnesium: 310-420 mg/day
Phosphorus: 700 mg/day
Potassium: 2,500-3,400 mg/day
Sodium: Less than 2,300 mg/day
Zinc: 8-11 mg/day
Iodine: 150 µg/day

Copper: 900 µg/day
Manganese: 1.8-2.3 mg/day
Fluoride: 3-4 mg/day
Antioxidants: Various amounts depending on the food source (e.g., flavonoids, polyphenols)

Phytochemicals: No specific RDI, but beneficial for health (e.g., carotenoids, flavonoids)

Probiotics: No specific RDI, but beneficial for gut health (e.g., from yogurt, kefir)